In [11]:
import json
import numpy as np
import pandas

#this code makes your editing experience using both python files 
#and notebooks a lot easier
%load_ext autoreload
#written to make things easier
%aimport utility 
%autoreload 1

all_sjc, all_state_appeals = utility.combine_cases()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
#returns case numbers and sections in which found within a case
def find_in_text(case_data,phrase):
    cases_found = []
    for case in range(len(case_data)):
        place_found = []
        for i in range(len(case_data[case]["text"])):
            if case_data[case]["text"][i].lower().find(phrase.lower()) != -1:
                place_found.append(i)
        
        if len(place_found) > 0:
            cases_found.append([case,place_found])
    return cases_found

In [13]:
def find_in_headnote(case_data,phrase):
    cases_found = []
    for case in range(len(case_data)):
        place_found = []
        for i in range(len(case_data[case]["headnote"])):
            if case_data[case]["headnote"][i].lower().find(phrase.lower()) != -1:
                place_found.append(i)
        
        if len(place_found) > 0:
            cases_found.append([case,place_found])
    return cases_found

In [14]:
# Searches list_of_keywords in string case_data
# and returns the total number of cases that contained any keyword
#as well as the "case numbers", that is, currently
#line numbers in the file
def number_of_cases_per_keywords(case_data, list_of_keywords):
    all_cases = set()
    for keyword in list_of_keywords:
        all_with_keyword = set()
        all_in_text = find_in_text(case_data, keyword)
        all_in_headnote = find_in_headnote(case_data, keyword)
        for occurence in all_in_text:
            all_with_keyword.add(occurence[0])
        for occurence in all_in_headnote:
            all_with_keyword.add(occurence[0])
        all_cases.update(all_with_keyword)
    pretty = list(all_cases)
    pretty.sort()
    return len(pretty), pretty

In [15]:
def contains(dataframe, str):
    return dataframe.text.str.contains(str)

#elementwise xor:
#returns true at i if and only if series1[i] and series2[i]
#are different
def logic_xor(series1, series2):
    return (series1 | series2) & ~(series1 & series2)

In [16]:
important_keywords = ["confidential informant", " ci ", "snitch", "informant", "gang informant"]

In [106]:
#Something weird is going on where there's duplicate indeces: I'm not sure how to handle this
print("statistics of how we are double counting: values represent how many times an index appears")
print(all_sjc.index.to_series().value_counts().describe(), "\n\n")
      
print("showing you in the other list, max is like half of the whole list (statistics a)")
print(all_state_appeals.index.to_series().describe(), "\n")

print("statistics of how bad the problem is in that list")
print(all_state_appeals.index.to_series().value_counts().describe(), "\n\n")

print("thus, if you sort things, it looks like this")
print(all_state_appeals.sort_index()[0:6], "\n")



statistics of how we are double counting: values represent how many times an index appears
count    2009.000000
mean        1.321553
std         0.467189
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         2.000000
dtype: float64 


showing you in the other list, max is like half of the whole list (statistics a)
count    4356.000000
mean     1104.379017
std       655.543994
min         0.000000
25%       544.000000
50%      1088.500000
75%      1633.000000
max      2440.000000
dtype: float64 

statistics of how bad the problem is in that list
count    2441.000000
mean        1.784515
std         0.411243
min         1.000000
25%         2.000000
50%         2.000000
75%         2.000000
max         2.000000
dtype: float64 


thus, if you sort things, it looks like this
                                                case  \
0    S T E P H A N I E     &   a n o t h e r   v ...   
0  CITY OF CAMBRIDGE vs. EDWARD RONALD BALDASARO ...   
1  JOHN 

In [115]:
print("this leads to the following problem:")
print(all_sjc.loc[ 0, : ].text)

this leads to the following problem:
0    The present case is the most recent in a serie...
0    COWIN, J. We granted the Commonwealth's applic...
Name: text, dtype: object


In [70]:
#something is seriously wrong with our algorithm somewhere in here
messed_up = all_state_appeals[1:2].text
print(messed_up.values)

print(messed_up.values[0])

['\n\nT\nh\ni\ns\n \nc\na\ns\ne\n \na\nr\ni\ns\ne\ns\n \nf\nr\no\nm\n \nt\nh\ne\n \ng\nr\na\nn\nt\n \no\nf\n \na\n \ng\nr\no\nu\nn\nd\nw\na\nt\ne\nr\n \nd\ni\ns\nc\nh\na\nr\ng\ne\n \np\ne\nr\nm\ni\nt\n \n(\np\ne\nr\nm\ni\nt\n)\n \nt\no\n \nt\nh\ne\n \nt\no\nw\nn\n \no\nf\n \nF\na\nl\nm\no\nu\nt\nh\n \n(\nt\no\nw\nn\n)\n \nf\no\nr\n \nt\nh\ne\n \nc\no\nn\ns\nt\nr\nu\nc\nt\ni\no\nn\n \na\nn\nd\n \no\np\ne\nr\na\nt\ni\no\nn\n \no\nf\n \na\n \nw\na\ns\nt\ne\nw\na\nt\ne\nr\n \nt\nr\ne\na\nt\nm\ne\nn\nt\n \nf\na\nc\ni\nl\ni\nt\ny\n \n(\nt\nr\ne\na\nt\nm\ne\nn\nt\n \nf\na\nc\ni\nl\ni\nt\ny\n)\n \na\nn\nd\n \nl\ne\na\nc\nh\n \nf\ni\ne\nl\nd\n.\n\n\nT\nh\ne\n \np\nl\na\ni\nn\nt\ni\nf\nf\ns\n \na\nr\ne\n \na\nb\nu\nt\nt\ne\nr\ns\n \nt\no\n \nt\nh\ne\n \ns\ni\nt\ne\n \no\nf\n \nt\nh\ne\n \np\nr\no\np\no\ns\ne\nd\n \nt\nr\ne\na\nt\nm\ne\nn\nt\n \nf\na\nc\ni\nl\ni\nt\ny\n \no\nr\n \nt\nh\ne\n \np\nr\no\np\no\ns\ne\nd\n \nl\ne\na\nc\nh\n \nf\ni\ne\nl\nd\n,\n \na\nn\nd\n \ns\no\nm\ne\n \no\nr\n \na\n

In [110]:

print("You see this makes things much more reasonable to remove \
newlines but it also squishes punctuation sometimes:\n")
print(messed_up.str.replace(pat="\n", repl="").iloc[0][0:500], "...\n")

print("This is a big problem in the state appeals cases. Statistics for newlines in state appeals:\n",
      all_state_appeals.text.str.count("\n").describe())

print("\nNote: there are no newlines in sjc cases: \n", all_sjc.text.str.count("\n").describe(), "\n")

#so, replace the text column to have it be good
all_state_appeals.text = all_state_appeals.text.str.replace(pat="\n", repl="")


You see this makes things much more reasonable to remove newlines but it also squishes punctuation sometimes:

This case arises from the grant of a groundwater discharge permit (permit) to the town of Falmouth (town) for the construction and operation of a wastewater treatment facility (treatment facility) and leach field.The plaintiffs are abutters to the site of the proposed treatment facility or the proposed leach field, and some or all of the plaintiffs rely on private water wells near the site of the proposed treatment facility or leach field as their sole source of drinking water. Acting on the pla ...

This is a big problem in the state appeals cases. Statistics for newlines in state appeals:
 count    4356.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: text, dtype: float64

Note: there are no newlines in sjc cases: 
 count    2655.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%       

In [111]:
affirmed_indeces = all_sjc.text.str.contains("affirm")
print(affirmed_indeces.describe())
vacate_indeces = all_sjc.text.str.contains("vacate") | contains(all_sjc,"vacated")
reversed_indeces = all_sjc.text.str.contains("reverse") | contains(all_sjc, "we reverse") 

both_reversed_and_affirmed = reversed_indeces & affirmed_indeces
print("\nreversed and affirmed\n", both_reversed_and_affirmed.value_counts())
print(both_reversed_and_affirmed.sort_values())
print(all_sjc.loc[ 681, : ].text)

print("\nonly one or the other appears:")
print(logic_xor(reversed_indeces, affirmed_indeces).value_counts())


count     2655
unique       2
top       True
freq      2008
Name: text, dtype: object

reversed and affirmed
 False    2096
True      559
Name: text, dtype: int64
0       False
1027    False
1028    False
1029    False
1031    False
        ...  
1267     True
1270     True
1272     True
1209     True
681      True
Name: text, Length: 2655, dtype: bool
SPINA, J. The defendants, brothers John Evans (John) and Jimmy Evans (Jimmy), were convicted of murder in the first degree on theories of extreme atrocity or cruelty and of deliberate premeditation, and illegal possession of handguns (G. L. c. 269, § 10 [a]), and ammunition (G. L. c. 269, § 10 [h]). John was also convicted of assault by means of a dangerous weapon, discharging a firearm within 500 feet of a building (G. L. c. 269, § 12E), and a number of motor vehicle and weapons violations. [Note 2] Both defendants filed motions for a new trial, and motions for funds for an investigator as to claims raised in their motions for a new tri

### searching for total number

In [20]:
print("total number of cases was: ", len(all_sjc) + len(all_state_appeals))

total number of cases was:  7011


In [88]:
result = number_of_cases_per_keywords(cases, important_keywords)
print(result)

(27, [25, 32, 36, 78, 91, 93, 97, 159, 187, 201, 309, 314, 390, 420, 425, 435, 438, 466, 477, 491, 504, 507, 551, 580, 587, 601, 620])


In [89]:
print(number_of_cases_per_keywords(appeals, important_keywords))

(70, [10, 26, 27, 77, 83, 96, 128, 150, 161, 178, 221, 223, 233, 236, 249, 251, 266, 310, 341, 396, 409, 411, 419, 424, 473, 481, 486, 487, 498, 579, 597, 659, 711, 728, 762, 774, 786, 793, 808, 817, 910, 930, 980, 1075, 1097, 1119, 1177, 1194, 1289, 1365, 1409, 1411, 1436, 1461, 1514, 1517, 1557, 1575, 1588, 1614, 1623, 1624, 1673, 1706, 1719, 1739, 1745, 1783, 1817, 1858])


In [90]:
important_keywords.remove("informant")
print(number_of_cases_per_keywords(cases, important_keywords)[0] + number_of_cases_per_keywords(cases, important_keywords)[0])

30
['confidential informant', ' ci ', 'snitch', 'gang informant']


### Searching terms in text then headnote for cases.csv

In [28]:
print(find_in_text(cases,"confidential informant"))

[[309, [2]], [390, [16]], [438, [2, 8]], [466, [46, 48, 49, 51, 55, 63, 74, 80]], [477, [9, 10]], [491, [5, 35, 105, 146, 147, 149, 161]], [551, [17, 81, 82]], [580, [2, 16, 158]], [601, [1, 5, 26]]]


In [29]:
print(find_in_headnote(cases,"confidential informant"))

[[466, [3]], [477, [2]], [491, [8]], [580, [4]]]


In [30]:
print(find_in_text(cases," ci "))

[]


In [31]:
print(find_in_headnote(cases," ci "))

[]


In [32]:
print(find_in_text(cases,"snitch"))

[[25, [10]], [78, [15]], [159, [11, 59]], [187, [16]], [420, [8]], [425, [51]]]


In [33]:
print(find_in_headnote(cases,"snitch"))

[]


In [34]:
print(find_in_text(cases,"informant"))

[[32, [35, 44, 45, 53]], [36, [48]], [91, [2, 24, 31, 32, 38, 46, 47, 48, 50, 51, 52, 57]], [93, [43]], [97, [5, 74, 75]], [201, [68, 70]], [309, [2]], [314, [20]], [390, [14, 16]], [435, [73]], [438, [2, 3, 4, 8, 10, 11, 19, 21]], [466, [2, 46, 48, 49, 51, 54, 55, 56, 63, 71, 72, 74, 77, 79, 80, 81, 84, 86, 88, 90]], [477, [9, 10, 15]], [491, [5, 35, 105, 146, 147, 149, 152, 159, 161]], [504, [110, 112, 113, 114, 119, 123, 124, 134, 151, 154, 155, 157]], [507, [123]], [551, [17, 81, 82]], [580, [2, 16, 120, 158]], [587, [60]], [601, [1, 4, 5, 26, 27, 30, 31, 39]], [620, [88]]]


In [35]:
print(find_in_headnote(cases,"informant"))

[[91, [2]], [466, [3]], [477, [2]], [491, [8]], [580, [4]]]


In [36]:
print(find_in_text(cases,"gang informant"))

[]


In [37]:
print(find_in_headnote(cases,"gang informant"))

[]


In [38]:
print(find_in_text(cases,"mafia informant"))

[]


In [39]:
print(find_in_headnote(cases,"mafia informant"))

[]


In [19]:
print(find_in_text(cases,"organized crime informant"))

[]


In [20]:
print(find_in_headnote(cases,"organized crime informant"))

[]


In [21]:
print(find_in_text(cases,"Stephen Lynch informant"))

[]


In [22]:
print(find_in_headnote(cases,"Stephen Lynch informant"))

[]


In [23]:
print(find_in_text(cases,"federal prison informant"))

[]


In [24]:
print(find_in_headnote(cases,"federal prison informant"))

[]


In [70]:
print(find_in_text(cases,"prison informant"))

[]


In [71]:
print(find_in_headnote(cases,"prison informant"))

[]


In [55]:
print(find_in_text(cases,"Whitey Bulger informant"))

[]


In [57]:
print(find_in_headnote(cases,"Whitey Bulger informant"))

[]


In [58]:
print(find_in_text(cases,"U.S. government informant"))

[]


In [61]:
print(find_in_headnote(cases,"U.S. government informant"))

[]


In [62]:
print(find_in_text(cases,"government informant"))

[[91, [38]]]


In [63]:
print(find_in_headnote(cases,"government informant"))

[]


In [67]:
print(find_in_text(cases,"government snitch"))

[]


In [68]:
print(find_in_headnote(cases,"government snitch"))

[]


### Searching terms in text then headnote for cases_appeals.csv

In [73]:
print(find_in_text(appeals,"confidential informant"))

[[10, [7]], [27, [8, 25, 36, 38]], [83, [7]], [96, [7]], [221, [2, 17, 19, 39]], [233, [1]], [249, [4, 6]], [266, [55, 58, 88, 94]], [341, [56]], [396, [4, 43]], [409, [27, 107]], [411, [7, 8, 9, 13, 23, 47, 49]], [424, [58]], [481, [3]], [486, [6, 24]], [487, [5]], [579, [66]], [786, [71]], [793, [2]], [817, [80]], [930, [10]], [980, [5]], [1097, [13, 20]], [1194, [32]], [1289, [10]], [1365, [1, 2]], [1409, [1, 7, 24, 34, 46]], [1411, [2, 3]], [1436, [6]], [1514, [3]], [1517, [51, 52]], [1588, [5]], [1623, [4, 38, 67, 69, 73, 83, 85, 86, 88, 91, 94, 97, 99]], [1624, [25]], [1673, [31, 32, 40, 41]], [1706, [9]], [1719, [59]], [1739, [7]], [1783, [1, 3, 17, 25, 32]], [1817, [6]]]


In [74]:
print(find_in_headnote(appeals,"confidential informant"))

[[27, [1]], [221, [1]], [486, [1]], [793, [2]], [1365, [1, 2]], [1411, [1]], [1588, [1]], [1623, [2]], [1673, [1]], [1783, [1]], [1817, [1]]]


In [107]:
print(find_in_text(appeals," ci "))

[[27, [8, 10, 14, 15, 16, 19, 29, 31, 36, 37, 41, 59, 61]], [233, [14, 16, 17, 20, 21, 22, 49, 51]], [396, [4, 7, 38, 46, 49]], [409, [27]], [487, [5, 6, 8, 9, 10, 11, 14, 22]], [793, [7, 9, 10, 12, 32, 50]], [1365, [2, 3, 5, 6, 17, 18, 24, 27, 28, 30, 33, 34, 36, 38, 57, 58]], [1436, [6]], [1623, [7, 8, 11, 23, 27, 35, 41]], [1673, [6, 7, 8, 10, 26, 39, 41, 46, 54]], [1739, [7, 12]]]


In [76]:
print(find_in_headnote(appeals," ci "))

[]


In [77]:
print(find_in_text(appeals,"snitch"))

[[150, [11]], [161, [14]], [310, [3, 4, 5, 15, 45, 46]], [762, [5, 30]], [808, [45]], [1858, [71]]]


In [78]:
print(find_in_headnote(appeals,"snitch"))

[]


In [109]:
print(find_in_text(appeals,"informant"))

[[10, [7]], [26, [21, 22, 27, 30, 31, 32, 33]], [27, [8, 25, 29, 34, 35, 36, 38, 41, 49, 55]], [77, [19]], [83, [7, 8, 9, 33]], [96, [7, 25, 35]], [128, [2, 70]], [178, [2, 7, 17, 19, 21, 27, 29, 31, 32, 83]], [221, [2, 4, 17, 19, 26, 27, 39, 42, 43, 44]], [223, [7]], [233, [1, 36, 52, 53, 57]], [236, [14, 16]], [249, [4, 6, 16, 20]], [251, [5]], [266, [55, 58, 63, 66, 67, 72, 85, 88, 89, 94]], [341, [56, 60, 61, 62, 65, 67]], [396, [4, 43]], [409, [27, 68, 76, 78, 84, 92, 96, 102, 107, 108, 113, 114]], [411, [7, 8, 9, 13, 14, 23, 47, 49]], [419, [22, 35]], [424, [58, 76, 78, 79, 82]], [473, [19]], [481, [3]], [486, [6, 24, 31, 52]], [487, [5, 18, 20, 41]], [498, [31, 33, 37]], [579, [8, 28, 29, 30, 32, 37, 41, 42, 51, 60, 61, 62, 66, 67]], [597, [47, 50, 54, 55, 56, 58]], [659, [1, 3, 22, 26, 34, 37]], [711, [1, 3, 10, 11, 20]], [728, [73, 94, 158, 159, 160, 167]], [774, [107]], [786, [71, 80, 90]], [793, [2, 44, 49]], [817, [80]], [910, [11, 16, 17]], [930, [2, 10, 11, 13, 30, 33, 36

In [80]:
print(find_in_headnote(appeals,"informant"))

[[27, [1]], [178, [1, 2]], [221, [1, 2]], [486, [1]], [579, [1]], [597, [1]], [711, [1, 2, 3]], [793, [2]], [930, [1]], [1097, [3]], [1365, [1, 2]], [1409, [1]], [1411, [1]], [1588, [1]], [1623, [2]], [1673, [1]], [1745, [2]], [1783, [1]], [1817, [1]]]


In [81]:
print(find_in_text(appeals,"gang informant"))

[]


In [82]:
print(find_in_headnote(appeals,"gang informant"))

[]


In [83]:
print(find_in_text(appeals,"mafia informant"))

[]


In [84]:
print(find_in_headnote(appeals,"mafia informant"))

[]


In [85]:
print(find_in_text(appeals,"organized crime informant"))

[]


In [86]:
print(find_in_headnote(appeals,"organized crime informant"))

[]


In [87]:
print(find_in_text(appeals,"Stephen Lynch informant"))

[]


In [88]:
print(find_in_headnote(appeals,"Stephen Lynch informant"))

[]


In [89]:
print(find_in_text(appeals,"federal prison informant"))

[]


In [90]:
print(find_in_headnote(appeals,"federal prison informant"))

[]


In [91]:
print(find_in_text(appeals,"prison informant"))

[]


In [92]:
print(find_in_headnote(appeals,"prison informant"))

[]


In [93]:
print(find_in_text(appeals,"Whitey Bulger informant"))

[]


In [94]:
print(find_in_headnote(appeals,"Whitey Bulger informant"))

[]


In [95]:
print(find_in_text(appeals,"U.S. government informant"))

[]


In [96]:
print(find_in_headnote(appeals,"U.S. government informant"))

[]


In [97]:
print(find_in_text(appeals,"government informant"))

[[178, [19]], [579, [66]], [711, [20]]]


In [98]:
print(find_in_headnote(appeals,"government informant"))

[[178, [1, 2]]]


In [99]:
print(find_in_text(appeals,"government snitch"))

[]


In [100]:
print(find_in_headnote(appeals,"government snitch"))

[]


In [120]:
print(len(find_in_text(cases,"appeal")))

626


In [121]:
len(cases)

646